In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [0]:
# creating vendor dimension
dim_vendor = spark.createDataFrame([
    (1, "Creative Mobile Technologies, LLC"),
    (2, "Curb Mobility, LLC"),
    (6, "Myle Technologies Inc")
], ["vendor_id", "vendor_name"])

dim_vendor.write.mode("overwrite").format("delta").saveAsTable("nyc_taxi.gold.dim_vendor")

In [0]:
# creating rate code dimension
dim_ratecode = spark.createDataFrame([
    (1, "Standard rate"),
    (2, "JFK"),
    (3, "Newark"),
    (4, "Nassau or Westchester"),
    (5, "Negotiated fare"),
    (6, "Group ride"),
    (99, "Null/Unknown")
], ["ratecode_id", "ratecode_description"])

dim_ratecode.write.mode("overwrite").format("delta").saveAsTable("nyc_taxi.gold.dim_ratecode")

In [0]:
# creating payment dimension
dim_payment = spark.createDataFrame([
    (0, "Flex Fare trip"),
    (1, "Credit card"),
    (2, "Cash"),
    (3, "No charge"),
    (4, "Dispute"),
    (5, "Unknown"),
    (6, "Voided trip")
], ["payment_type", "payment_description"])

dim_payment.write.mode("overwrite").format("delta").saveAsTable("nyc_taxi.gold.dim_payment_type")

In [0]:
# creating trip type dimension
dim_trip_type = spark.createDataFrame([
    (1, "Street-hail"),
    (2, "Dispatch")
], ["trip_type", "trip_type_description"])

dim_trip_type.write.mode("overwrite").format("delta").saveAsTable("nyc_taxi.gold.dim_trip_type")

In [0]:
# creating zone dimension
df_zone = spark.read.csv("gs://nyc_taxi_analysis_project/taxi_zone_lookup.csv", header=True, inferSchema=True)
df_zone = df_zone.withColumnRenamed("LocationID", "location_id") \
    .withColumnRenamed("Borough", "borough") \
    .withColumnRenamed("Zone", "zone") \
    .withColumnRenamed("service_zone", "service_zone")
df_zone.write.mode("overwrite").format("delta").saveAsTable("nyc_taxi.gold.dim_location")

### **FACT MODELLING**

In [0]:
df_silver = spark.table("nyc_taxi.silver.data")

In [0]:
df_fact=df_silver.drop('source_file','ingestion_time')

In [0]:
df_fact.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("pickup_year", "pickup_month") \
    .saveAsTable("nyc_taxi.gold.fact_trips")